## BigDL-Nano example on Stanford Dogs dataset
---
This example illustrates how to apply bigdl-nano optimizations on a image recognition case based on Tensorflow Keras framework. The basic image recognition module is pre-trained EfficientNetB0 from tensorflow.keras.applications and fine-tune it on [Stanford Dogs](http://vision.stanford.edu/aditya86/ImageNetDogs/) image recognition dataset. 

In [1]:
import os
from time import time

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0
import tensorflow_datasets as tfds

from bigdl.nano.tf.keras import Model

In [2]:
IMG_SIZE=224
BATCH_SIZE=int(os.environ.get('BATCH_SIZE', 64))
DATASET_NAME="stanford_dogs"

### Loading data
---
Here we load data from tensorflow_datasets (hereafter TFDS). Stanford Dogs dataset is provided in TFDS as stanford_dogs. It features 20,580 images that belong to 120 classes of dog breeds (12,000 for training and 8,580 for testing).

In [3]:
(ds_train, ds_test), ds_info=tfds.load(
    DATASET_NAME, data_dir="tensorflow_datasets/" ,split=["train", "train"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes
FIT_STEPS = int(os.environ.get('FIT_STEPS', len(ds_train)/BATCH_SIZE))
VAL_STEPS = int(os.environ.get('VAL_STEPS')) if 'VAL_STEPS' in os.environ else None
FREEZE_EPOCHS = int(os.environ.get('FREEZE_EPOCHS', 25)) # @param {type: "slider", min:8, max:80}
UNFREEZE_EPOCHS = int(os.environ.get('UNFREEZE_EPOCHS', 10)) # @param {type: "slider", min:8, max:50}
NUM_PROCESSES = int(os.environ.get('NUM_PROCESSES', 4))

size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

2022-06-10 03:29:42.207687: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data augmentation
---
To augment the dataset it can beneficial to make augmenter functions: a function that receives an image (a tf.Tensor) and a label and returns a new augmented image and label. By defining functions for each augmentation operation we can easily attach them to datasets and control when they are evaluated.

In [4]:
def rotate(image, label):
    return (tf.image.rot90(image, tf.random_uniform_initializer(minval=0, maxval=4)(shape=[], dtype=tf.int32)), label)

def flip(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return (image, label)

def contrast(image, label):
    return (tf.image.random_contrast(image, 0.7, 1.3), label)

def color(image, label):
    image = tf.image.random_hue(image, 0.08)
    image = tf.image.random_saturation(image, 0.6, 1.6)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, 0.7, 1.3)
    return (image, label)


In [5]:
ds_train = ds_train.repeat(50)
augmentations = [rotate, flip, contrast, color]

for f in augmentations:
    ds_train = ds_train.map(f, num_parallel_calls=4)

ds_train = ds_train.shuffle(1000)

### Prepare inputs
---
Once we verify the input data and augmentation are working correctly, we prepare dataset for training. The input data are resized to uniform IMG_SIZE. The labels are put into one-hot (a.k.a. categorical) encoding. The dataset is batched.

In [6]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=BATCH_SIZE, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=BATCH_SIZE, drop_remainder=True)

ds_train = ds_train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
ds_test = ds_test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Transfer learning from pre-trained weights
---
Here we initialize the model with pre-trained ImageNet weights, and we fine-tune it on Stanford Dogs dataset.

In [7]:
def build_model(num_classes, learning_rate=1e-2):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

### Train
---
Use Model.fit from bigdl.nano.tf.keras for BigDl-Nano tf.keras.

This function override tf.keras.Model.fit to add more parameters.


Additional parameters:
```
        :param num_processes:  when num_processes is not None, it specifies how many sub-processes
                               to launch to run pseudo-distributed training; when num_processes is None,
                               training will run in the current process.
                               
        :param backend: Use backend 'multiprocessing', 'horovod', 'ray', defaults to None.
                        when num_processes is not None, it specifies which backend to use when
                       launching sub-processes to run psedu-distributed training; 
                       when num_processes is None, this parameter takes no effect.
```

### Single process

In [8]:
model_single = build_model(num_classes=NUM_CLASSES)

start = time()
model_single.fit(ds_train,
                 epochs=FREEZE_EPOCHS,
                 steps_per_epoch=FIT_STEPS,
                 validation_data=ds_test,
                 validation_steps=VAL_STEPS,
                 verbose=1)

Epoch 1/25


tcmalloc: large alloc 1073741824 bytes == 0x5559d20b4000 @  0x7fa596addd3f 0x7fa596b140c0 0x7fa596b17082 0x7fa596b17243 0x7fa587148402 0x7fa57b4e2eb0 0x7fa57b5030b5 0x7fa57b5069ea 0x7fa57b506f69 0x7fa57b5072d1 0x7fa57b4fbce3 0x7fa576bc1051 0x7fa576a1c38d 0x7fa5767af087 0x7fa5767af91e 0x7fa5767afb1d 0x7fa57f702c92 0x7fa57f71b6d8 0x7fa57f76fe3e 0x7fa576bc2d7c 0x7fa576b4ccec 0x7fa57bf0f76e 0x7fa57bf0c1f3 0x7fa57729e313 0x7fa596a73609 0x7fa596998133


  6/187 [..............................] - ETA: 1:10 - loss: 5.4018 - accuracy: 0.0885

tcmalloc: large alloc 2147483648 bytes == 0x555a1531a000 @  0x7fa596addd3f 0x7fa596b140c0 0x7fa596b17082 0x7fa596b17243 0x7fa587148402 0x7fa57b4e2eb0 0x7fa57b5030b5 0x7fa57b5069ea 0x7fa57b506f69 0x7fa57b5072d1 0x7fa57b4fbce3 0x7fa576bc1051 0x7fa576a1c38d 0x7fa5767af087 0x7fa5767af91e 0x7fa5767afb1d 0x7fa5821a0bf5 0x7fa576bc2d7c 0x7fa576b4ccec 0x7fa57bf0f76e 0x7fa57bf0c1f3 0x7fa57729e313 0x7fa596a73609 0x7fa596998133


 61/187 [========>.....................] - ETA: 47s - loss: 4.1153 - accuracy: 0.3010

...

tcmalloc: large alloc 68719476736 bytes == 0x556a1bc90000 @  0x7fa596addd3f 0x7fa596b140c0 0x7fa596b17082 0x7fa596b17243 0x7fa587148402 0x7fa57b4e2eb0 0x7fa57b5030b5 0x7fa57b5069ea 0x7fa57b506f69 0x7fa57b5072d1 0x7fa57b4fbce3 0x7fa576bc1051 0x7fa576a1c38d 0x7fa5767af087 0x7fa5767af91e 0x7fa5767afb1d 0x7fa5821a0bf5 0x7fa576bc2d7c 0x7fa576b4ccec 0x7fa57bf0f76e 0x7fa57bf0c1f3 0x7fa57729e313 0x7fa596a73609 0x7fa596998133


187/187 [==============================] - 114s 611ms/step - loss: 1.2493 - accuracy: 0.6385 - val_loss: 0.3974 - val_accuracy: 0.8736
Epoch 10/25
187/187 [==============================] - 113s 604ms/step - loss: 1.2304 - accuracy: 0.6528 - val_loss: 0.3709 - val_accuracy: 0.8747
Epoch 11/25
187/187 [==============================] - 112s 601ms/step - loss: 1.2482 - accuracy: 0.6461 - val_loss: 0.4429 - val_accuracy: 0.8572
Epoch 12/25
187/187 [==============================] - 113s 603ms/step - loss: 1.2149 - accuracy: 0.6584 - val_loss: 0.3909 - val_accuracy: 0.8722
Epoch 13/25
187/187 [==============================] - 113s 603ms/step - loss: 1.2142 - accuracy: 0.6510 - val_loss: 0.3705 - val_accuracy: 0.8786
Epoch 14/25
187/187 [==============================] - 112s 602ms/step - loss: 1.2133 - accuracy: 0.6555 - val_loss: 0.3943 - val_accuracy: 0.8698
Epoch 15/25
187/187 [==============================] - 113s 606ms/step - loss: 1.2250 - accuracy: 0.6579 - val_loss: 0.3859 - val_

tcmalloc: large alloc 137438953472 bytes == 0x557a9fc90000 @  0x7fa596addd3f 0x7fa596b140c0 0x7fa596b17082 0x7fa596b17243 0x7fa587148402 0x7fa57b4e2eb0 0x7fa57b5030b5 0x7fa57b5069ea 0x7fa57b506f69 0x7fa57b5072d1 0x7fa57b4fbce3 0x7fa576bc1051 0x7fa576a1c38d 0x7fa5767af087 0x7fa5767af91e 0x7fa5767afb1d 0x7fa5821a0bf5 0x7fa576bc2d7c 0x7fa576b4ccec 0x7fa57bf0f76e 0x7fa57bf0c1f3 0x7fa57729e313 0x7fa596a73609 0x7fa596998133
tcmalloc: large alloc 4294967296 bytes == 0x559aa7c90000 @  0x7fa596addd3f 0x7fa596b13ba6 0x7fa596b16a5a 0x7fa57b4fd836 0x7fa57b502f72 0x7fa57b5069ea 0x7fa57b506f69 0x7fa57b5072d1 0x7fa57b4fbce3 0x7fa576bc1051 0x7fa576a1c38d 0x7fa5767af087 0x7fa5767af91e 0x7fa5767afb1d 0x7fa5821a0bf5 0x7fa576bc2d7c 0x7fa576b4ccec 0x7fa57bf0f76e 0x7fa57bf0c1f3 0x7fa57729e313 0x7fa596a73609 0x7fa596998133


187/187 [==============================] - 113s 604ms/step - loss: 1.2136 - accuracy: 0.6588 - val_loss: 0.3783 - val_accuracy: 0.8768
Epoch 19/25
187/187 [==============================] - 112s 598ms/step - loss: 1.2199 - accuracy: 0.6616 - val_loss: 0.3519 - val_accuracy: 0.8844
Epoch 20/25
187/187 [==============================] - 111s 596ms/step - loss: 1.2443 - accuracy: 0.6574 - val_loss: 0.4113 - val_accuracy: 0.8639
Epoch 21/25
187/187 [==============================] - 112s 599ms/step - loss: 1.2092 - accuracy: 0.6603 - val_loss: 0.3787 - val_accuracy: 0.8752
Epoch 22/25
187/187 [==============================] - 111s 597ms/step - loss: 1.2223 - accuracy: 0.6600 - val_loss: 0.4108 - val_accuracy: 0.8590
Epoch 23/25
187/187 [==============================] - 112s 600ms/step - loss: 1.2001 - accuracy: 0.6695 - val_loss: 0.3748 - val_accuracy: 0.8765
Epoch 24/25
187/187 [==============================] - 112s 602ms/step - loss: 1.2118 - accuracy: 0.6621 - val_loss: 0.3680 - val_

2022-06-10 04:16:55.724107: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
unfreeze_model(model_single)
model_single.fit(ds_train,
                 epochs=UNFREEZE_EPOCHS,
                 steps_per_epoch=FIT_STEPS,
                 validation_data=ds_test,
                 validation_steps=VAL_STEPS,
                 verbose=1)

Epoch 1/10
187/187 [==============================] - 142s 728ms/step - loss: 0.9584 - accuracy: 0.7172 - val_loss: 0.2450 - val_accuracy: 0.9236
Epoch 2/10
187/187 [==============================] - 137s 733ms/step - loss: 0.8080 - accuracy: 0.7550 - val_loss: 0.2186 - val_accuracy: 0.9311
Epoch 3/10
187/187 [==============================] - 136s 728ms/step - loss: 0.7679 - accuracy: 0.7705 - val_loss: 0.2000 - val_accuracy: 0.9360
Epoch 4/10
187/187 [==============================] - 136s 729ms/step - loss: 0.6943 - accuracy: 0.7903 - val_loss: 0.1861 - val_accuracy: 0.9413
Epoch 5/10
187/187 [==============================] - 136s 727ms/step - loss: 0.6795 - accuracy: 0.7868 - val_loss: 0.1737 - val_accuracy: 0.9459
Epoch 6/10
187/187 [==============================] - 135s 723ms/step - loss: 0.6311 - accuracy: 0.7996 - val_loss: 0.1630 - val_accuracy: 0.9477
Epoch 7/10
187/187 [==============================] - 136s 728ms/step - loss: 0.5861 - accuracy: 0.8187 - val_loss: 0.1462 -

2022-06-10 04:39:37.991699: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [10]:
fit_time_model_single = time() - start
acc_model_single = model_single.evaluate(ds_test, steps=VAL_STEPS, verbose=1)
model_single.save("EfficientNetB0.h5")

187/187 [==============================] - 54s 288ms/step - loss: 0.1264 - accuracy: 0.9600


/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


### Multiple processes

In [8]:
model_multiple = build_model(num_classes=NUM_CLASSES, learning_rate=1e-2)

start = time()
model_multiple.fit(ds_train,
                   epochs=FREEZE_EPOCHS,
                   validation_data=ds_test,
                   steps_per_epoch=FIT_STEPS,
                   validation_steps=VAL_STEPS,
                   verbose=1,
                   num_processes=NUM_PROCESSES,
                   backend="multiprocessing")

tcmalloc: large alloc 1073741824 bytes == 0x55a042126000 @  0x7f56a5f2dd3f 0x7f56a5f640c0 0x7f56a5f67082 0x7f56a5f67243 0x7f5696598402 0x7f568a932eb0 0x7f568a9530b5 0x7f568a9569ea 0x7f568a956f69 0x7f568a9572d1 0x7f568a94bce3 0x7f5686011051 0x7f5685e6c38d 0x7f5685bff087 0x7f5685bff91e 0x7f5685bffb1d 0x7f568eb52c92 0x7f568eb6b6d8 0x7f568ebbfe3e 0x7f5686012d7c 0x7f5685f9ccec 0x7f568b35f76e 0x7f568b35c1f3 0x7f56866ee313 0x7f56a5ec3609 0x7f56a5de8133
2022-06-09 11:08:50.070695: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-06-09 11:08:57.448326: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may ch

INFO:tensorflow:Assets written to: /tmp/tmpm1o1ebux/temp_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpm1o1ebux/temp_model/assets
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022-06-09 11:09:16.218087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild Te

Epoch 1/25
 73/187 [==========>...................] - ETA: 30s - loss: 4.1473 - accuracy: 0.2964

tcmalloc: large alloc 2147483648 bytes == 0x55a0717c2000 @  0x7f30c2cead3f 0x7f30c2d210c0 0x7f30c2d24082 0x7f30c2d24243 0x7f30bccc6402 0x7f30b1060eb0 0x7f30b10810b5 0x7f30b10849ea 0x7f30b1084f69 0x7f30b10852d1 0x7f30b1079ce3 0x7f30ac73f051 0x7f30ac59a38d 0x7f30ac32d087 0x7f30ac32d91e 0x7f30ac32db1d 0x7f30b230dbfd 0x7f30ac740d7c 0x7f30ac6c76d5 0x7f30ac6bb24e 0x7f30b1a8d76e 0x7f30b1a8a1f3 0x7f30ace1c313 0x7f30c2c80609 0x7f30c2ba5133
tcmalloc: large alloc 2147483648 bytes == 0x555f3be64000 @  0x7fdb65b89d3f 0x7fdb65bc00c0 0x7fdb65bc3082 0x7fdb65bc3243 0x7fdb5fb65402 0x7fdb53effeb0 0x7fdb53f200b5 0x7fdb53f239ea 0x7fdb53f23f69 0x7fdb53f242d1 0x7fdb53f18ce3 0x7fdb4f5de051 0x7fdb4f43938d 0x7fdb4f1cc087 0x7fdb4f1cc91e 0x7fdb4f1ccb1d 0x7fdb551acbfd 0x7fdb4f5dfd7c 0x7fdb4f5666d5 0x7fdb4f55a24e 0x7fdb5492c76e 0x7fdb549291f3 0x7fdb4fcbb313 0x7fdb65b1f609 0x7fdb65a44133


...

2022-06-09 11:25:23.233629: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:25:23.233728: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:25:23.235472: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:25:23.236030: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


Epoch 25/25
187/187 [==============================] - ETA: 0s - loss: 1.3422 - accuracy: 0.6319

2022-06-09 11:26:00.117775: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:26:00.120057: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:26:00.130669: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:26:00.134641: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:26:00.159587: W tensorflow/core/kernels/data/cache_dat

187/187 [==============================] - 37s 197ms/step - loss: 1.3422 - accuracy: 0.6319


2022-06-09 11:26:09.235908: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-09 11:26:09.288925: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-09 11:26:09.379701: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-09 11:26:09.545982: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_l

In [9]:
unfreeze_model(model_multiple)

model_multiple.fit(ds_train,
                  epochs=UNFREEZE_EPOCHS,
                   steps_per_epoch=FIT_STEPS,
                  validation_data=ds_test,
                   validation_steps=VAL_STEPS,
                  verbose=1,
                  num_processes=NUM_PROCESSES,
                  backend="multiprocessing")

2022-06-09 11:27:18.083041: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


INFO:tensorflow:Assets written to: /tmp/tmpx6b3acmy/temp_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpx6b3acmy/temp_model/assets
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022-06-09 11:27:45.785284: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild Te

Epoch 1/10
 73/187 [==========>...................] - ETA: 33s - loss: 1.2107 - accuracy: 0.6629

tcmalloc: large alloc 2147483648 bytes == 0x55a06f3ea000 @  0x7f0968b63d3f 0x7f0968b9a0c0 0x7f0968b9d082 0x7f0968b9d243 0x7f0962b3f402 0x7f0956ed9eb0 0x7f0956efa0b5 0x7f0956efd9ea 0x7f0956efdf69 0x7f0956efe2d1 0x7f0956ef2ce3 0x7f09525b8051 0x7f095241338d 0x7f09521a6087 0x7f09521a691e 0x7f09521a6b1d 0x7f0958186bfd 0x7f09525b9d7c 0x7f09525406d5 0x7f095253424e 0x7f095790676e 0x7f09579031f3 0x7f0952c95313 0x7f0968af9609 0x7f0968a1e133


...

2022-06-09 11:35:06.053245: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:06.059118: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:06.060861: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:06.072665: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


Epoch 9/10
187/187 [==============================] - ETA: 0s - loss: 0.6822 - accuracy: 0.7888

2022-06-09 11:35:52.018861: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:52.021350: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:52.025506: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:35:52.026554: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


187/187 [==============================] - 46s 246ms/step - loss: 0.6822 - accuracy: 0.7888
Epoch 10/10
187/187 [==============================] - 45s 240ms/step - loss: 0.6680 - accuracy: 0.7949


2022-06-09 11:36:36.809924: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:36:36.810250: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:36:36.814418: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:36:36.817421: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-06-09 11:36:36.849881: W tensorflow/core/kernels/data/cache_dat

In [10]:
fit_time_model_multiple = time() - start
acc_model_multiple = model_multiple.evaluate(ds_test, steps=VAL_STEPS, verbose=1)

 20/187 [==>...........................] - ETA: 45s - loss: 0.1617 - accuracy: 0.9477

tcmalloc: large alloc 2147483648 bytes == 0x55a12eb7e000 @  0x7f56a5f2dd3f 0x7f56a5f640c0 0x7f56a5f67082 0x7f56a5f67243 0x7f5696598402 0x7f568a932eb0 0x7f568a9530b5 0x7f568a9569ea 0x7f568a956f69 0x7f568a9572d1 0x7f568a94bce3 0x7f5686011051 0x7f5685e6c38d 0x7f5685bff087 0x7f5685bff91e 0x7f5685bffb1d 0x7f56915f0bf5 0x7f5686012d7c 0x7f5685f9ccec 0x7f568b35f76e 0x7f568b35c1f3 0x7f56866ee313 0x7f56a5ec3609 0x7f56a5de8133


 76/187 [===========>..................] - ETA: 27s - loss: 0.1697 - accuracy: 0.9474

tcmalloc: large alloc 4294967296 bytes == 0x55a1b337e000 @  0x7f56a5f2dd3f 0x7f56a5f640c0 0x7f56a5f67082 0x7f56a5f67243 0x7f5696598402 0x7f568a932eb0 0x7f568a9530b5 0x7f568a9569ea 0x7f568a956f69 0x7f568a9572d1 0x7f568a94bce3 0x7f5686011051 0x7f5685e6c38d 0x7f5685bff087 0x7f5685bff91e 0x7f5685bffb1d 0x7f56915f0bf5 0x7f5686012d7c 0x7f5685f9ccec 0x7f568b35f76e 0x7f568b35c1f3 0x7f56866ee313 0x7f56a5ec3609 0x7f56a5de8133


187/187 [==============================] - 45s 237ms/step - loss: 0.1595 - accuracy: 0.9509


### Multiple processes with horovod

In [14]:
model_multiple_horovod = build_model(num_classes=NUM_CLASSES, learning_rate=1e-2)

start = time()
model_multiple_horovod.fit(ds_train,
                           epochs=FREEZE_EPOCHS,
                           validation_data=ds_test,
                           steps_per_epoch=FIT_STEPS,
                           validation_steps=VAL_STEPS,
                           verbose=1,
                           num_processes=NUM_PROCESSES,
                           backend="horovod")

2022-06-09 11:47:21.371783: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


INFO:tensorflow:Assets written to: /tmp/tmp0_67z5ql/temp_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp0_67z5ql/temp_model/assets
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
[3]<stderr>:2022-06-09 11:47:53.830895: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
[2]<stderr>:2022-06-09 11:47:53.83

[0]<stdout>:Epoch 1/25


[3]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x55e76e50a000 @  0x7f8aafacdd3f 0x7f8aafb040c0 0x7f8aafb07082 0x7f8aafb07243 0x7f8aa9abd402 0x7f8a9de57eb0 0x7f8a9de780b5 0x7f8a9de7b9ea 0x7f8a9de7bf69 0x7f8a9de7c2d1 0x7f8a9de70ce3 0x7f8a99536051 0x7f8a9939138d 0x7f8a99124087 0x7f8a9912491e 0x7f8a99124b1d 0x7f8aa2077c92 0x7f8aa20906d8 0x7f8aa20e4e3e 0x7f8a99537d7c 0x7f8a994c1cec 0x7f8a9e88476e 0x7f8a9e8811f3 0x7f8a99c13313 0x7f8aafa63609 0x7f8aaf988133
[2]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x556d23f72000 @  0x7f6dfe811d3f 0x7f6dfe8480c0 0x7f6dfe84b082 0x7f6dfe84b243 0x7f6df8801402 0x7f6decb9beb0 0x7f6decbbc0b5 0x7f6decbbf9ea 0x7f6decbbff69 0x7f6decbc02d1 0x7f6decbb4ce3 0x7f6de827a051 0x7f6de80d538d 0x7f6de7e68087 0x7f6de7e6891e 0x7f6de7e68b1d 0x7f6df0dbbc92 0x7f6df0dd46d8 0x7f6df0e28e3e 0x7f6de827bd7c 0x7f6de8205cec 0x7f6ded5c876e 0x7f6ded5c51f3 0x7f6de8957313 0x7f6dfe7a7609 0x7f6dfe6cc133
[1]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x55e5fcaee000 @  0x7f

...

[2]<stderr>:tcmalloc: large alloc 34359738368 bytes == 0x557527e66000 @  0x7f6dfe811d3f 0x7f6dfe8480c0 0x7f6dfe84b082 0x7f6dfe84b243 0x7f6df8801402 0x7f6decb9beb0 0x7f6decbbc0b5 0x7f6decbbf9ea 0x7f6decbbff69 0x7f6decbc02d1 0x7f6decbb4ce3 0x7f6de827a051 0x7f6de80d538d 0x7f6de7e68087 0x7f6de7e6891e 0x7f6de7e68b1d 0x7f6dede48bfd 0x7f6de827bd7c 0x7f6de8205cec 0x7f6ded5c876e 0x7f6ded5c51f3 0x7f6de8957313 0x7f6dfe7a7609 0x7f6dfe6cc133


[0]<stdout>:187/187 [==============================] - 64s 318ms/step - loss: 1.2791 - accuracy: 0.6494 - val_loss: 0.4043 - val_accuracy: 0.8702
[0]<stdout>:Epoch 19/25
[0]<stdout>:187/187 [==============================] - 62s 329ms/step - loss: 1.3400 - accuracy: 0.6190 - val_loss: 0.4060 - val_accuracy: 0.8672
[0]<stdout>:Epoch 20/25
[0]<stdout>:187/187 [==============================] - 60s 320ms/step - loss: 1.2716 - accuracy: 0.6487 - val_loss: 0.4254 - val_accuracy: 0.8696
[0]<stdout>:Epoch 21/25
[0]<stdout>:187/187 [==============================] - 60s 317ms/step - loss: 1.3254 - accuracy: 0.6267 - val_loss: 0.4306 - val_accuracy: 0.8570
[0]<stdout>:Epoch 22/25
[0]<stdout>:187/187 [==============================] - 62s 322ms/step - loss: 1.2847 - accuracy: 0.6447 - val_loss: 0.4079 - val_accuracy: 0.8624
[0]<stdout>:Epoch 23/25
[0]<stdout>:187/187 [==============================] - 60s 312ms/step - loss: 1.3003 - accuracy: 0.6434 - val_loss: 0.3634 - val_accuracy: 0.8736
[0]<

[1]<stderr>:2022-06-09 12:13:18.778839: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[0]<stdout>:187/187 [==============================] - 62s 316ms/step - loss: 1.2888 - accuracy: 0.6434 - val_loss: 0.3735 - val_accuracy: 0.8781


[0]<stderr>:2022-06-09 12:13:20.700714: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
[2]<stderr>:2022-06-09 12:13:20.893522: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
[3]<stderr>:2022-06-09 12:13:21.217032: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fu

In [15]:
unfreeze_model(model_multiple_horovod)

model_multiple_horovod.fit(ds_train,
                  epochs=UNFREEZE_EPOCHS,
                  steps_per_epoch=FIT_STEPS,
                  validation_data=ds_test,
                  validation_steps=VAL_STEPS,
                  verbose=1,
                  num_processes=NUM_PROCESSES,
                  backend="horovod")

2022-06-09 12:14:38.789260: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


INFO:tensorflow:Assets written to: /tmp/tmps4khrapt/temp_model/assets


INFO:tensorflow:Assets written to: /tmp/tmps4khrapt/temp_model/assets
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/envs/nanoWithTf/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
[3]<stderr>:2022-06-09 12:15:09.702898: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
[3]<stderr>:To enable them in othe

[0]<stdout>:Epoch 1/10


[3]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x562a35e96000 @  0x7faadfebbd3f 0x7faadfef20c0 0x7faadfef5082 0x7faadfef5243 0x7faad9eab402 0x7faace245eb0 0x7faace2660b5 0x7faace2699ea 0x7faace269f69 0x7faace26a2d1 0x7faace25ece3 0x7faac9924051 0x7faac977f38d 0x7faac9512087 0x7faac951291e 0x7faac9512b1d 0x7faad2465c92 0x7faad247e6d8 0x7faad24d2e3e 0x7faac9925d7c 0x7faac98afcec 0x7faacec7276e 0x7faacec6f1f3 0x7faaca001313 0x7faadfe51609 0x7faadfd76133
[1]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x55dfd1478000 @  0x7f8040ff3d3f 0x7f804102a0c0 0x7f804102d082 0x7f804102d243 0x7f803afe3402 0x7f802f37deb0 0x7f802f39e0b5 0x7f802f3a19ea 0x7f802f3a1f69 0x7f802f3a22d1 0x7f802f396ce3 0x7f802aa5c051 0x7f802a8b738d 0x7f802a64a087 0x7f802a64a91e 0x7f802a64ab1d 0x7f803359dc92 0x7f80335b66d8 0x7f803360ae3e 0x7f802aa5dd7c 0x7f802a9e7cec 0x7f802fdaa76e 0x7f802fda71f3 0x7f802b139313 0x7f8040f89609 0x7f8040eae133
[2]<stderr>:tcmalloc: large alloc 1073741824 bytes == 0x556a14076000 @  0x7f

...

[3]<stderr>:tcmalloc: large alloc 17179869184 bytes == 0x562e1948c000 @  0x7faadfebbd3f 0x7faadfef20c0 0x7faadfef5082 0x7faadfef5243 0x7faad9eab402 0x7faace245eb0 0x7faace2660b5 0x7faace2699ea 0x7faace269f69 0x7faace26a2d1 0x7faace25ece3 0x7faac9924051 0x7faac977f38d 0x7faac9512087 0x7faac951291e 0x7faac9512b1d 0x7faacf4f2bfd 0x7faac9925d7c 0x7faac98afcec 0x7faacec7276e 0x7faacec6f1f3 0x7faaca001313 0x7faadfe51609 0x7faadfd76133


[0]<stdout>:187/187 [==============================] - 79s 420ms/step - loss: 0.7044 - accuracy: 0.7931 - val_loss: 0.1730 - val_accuracy: 0.9484
[0]<stdout>:Epoch 9/10
[0]<stdout>:187/187 [==============================] - 78s 415ms/step - loss: 0.6612 - accuracy: 0.7914 - val_loss: 0.1640 - val_accuracy: 0.9477
[0]<stdout>:Epoch 10/10


[1]<stderr>:2022-06-09 12:28:24.106138: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
[2]<stderr>:2022-06-09 12:28:24.789655: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[0]<stdout>:187/187 [==============================] - 78s 413ms/step - loss: 0.6321 - accuracy: 0.8021 - val_loss: 0.1503 - val_accuracy: 0.9579


[0]<stderr>:2022-06-09 12:28:31.470305: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
[3]<stderr>:2022-06-09 12:28:31.847628: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
[1]<stderr>:2022-06-09 12:28:34.064152: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but t

In [16]:
fit_time_model_multiple_horovod = time() - start
acc_model_multiple_horovod = model_multiple_horovod.evaluate(ds_test, steps=VAL_STEPS, verbose=1)

187/187 [==============================] - 43s 227ms/step - loss: 0.1593 - accuracy: 0.9518


In [19]:
template = """
|        Precision     | Fit Time(s)       | Accuracy(%) |
|         Single       |       {:5.2f}       |    {:5.2f}    |
|        Multiple      |       {:5.2f}       |    {:5.2f}    |
| Multiple With Horovod|       {:5.2f}       |    {:5.2f}    |
"""
summary = template.format(
    fit_time_model_single, acc_model_single[1] * 100,
    fit_time_model_multiple, acc_model_multiple[1] * 100,
    fit_time_model_multiple_horovod, acc_model_multiple_horovod[1] * 100
)
print(summary)


|        Precision     | Fit Time(s)       | Accuracy(%) |
|         Single       |       4159.40       |    96.11    |
|        Multiple      |       1746.63       |    95.09    |
| Multiple With Horovod|       2548.35       |    95.18    |

